In [ ]:
# Copyright © 2025 Preetimant Bora Bhowal. All rights reserved.
# Unauthorized copying, distribution, or use of this file is strictly prohibited.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json
import uuid
import shutil
import re
from datetime import datetime
from collections import defaultdict

In [ ]:
def generate_dialogflow_agent(input_folder, output_folder):
    temp_dir = os.path.join(output_folder, "dialogflow_agent")
    intents_dir = os.path.join(temp_dir, "intents")
    entities_dir = os.path.join(temp_dir, "entities")

    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(intents_dir)
    os.makedirs(entities_dir)

    entity_data = defaultdict(lambda: defaultdict(set))
    entity_data['CourseInfo']['synonyms'] = defaultdict(list)
    all_assessments = set()
    all_instructors = set()
    course_properties = set()

    # Process course files
    for filename in os.listdir(input_folder):
        if filename.endswith(".json"):
            filepath = os.path.join(input_folder, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                try:
                    course = json.load(f)
                    process_course(course, entity_data, all_assessments, all_instructors, course_properties)
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")
                    continue

    # Create entities with enhanced ontology relationships
    create_entities(entities_dir, entity_data, all_assessments, all_instructors, course_properties)

    # Create intents with ontology-aware structure
    create_intents(intents_dir)

    # Create metadata with webhook enablement
    create_metadata(temp_dir)

    # Package agent
    zip_path = os.path.join(output_folder, "CourseSensei(v2.1)_$intents.zip")
    shutil.make_archive(zip_path.replace(".zip", ""), "zip", temp_dir)
    shutil.rmtree(temp_dir)
    print(f"Agent package created: {zip_path}")

def generate_abbreviation(course_name):
    # Step 1: Extract potential acronyms from brackets
    bracket_pattern = re.compile(r'\(([A-Z0-9]{2,5})\)|\[([A-Z0-9]{2,5})\]|\{([A-Z0-9]{2,5})\}')
    matches = bracket_pattern.findall(course_name)

    # Prioritize acronyms in order: () → [] → {}
    for group in matches:
        acronym = next((m for m in group if m), None)
        if acronym and 2 <= len(acronym) <= 5:  # Valid acronym length
            return acronym

    # Step 2: If no acronym found, proceed with generated abbreviation
    # Remove course codes (e.g., OQM404), parenthetical content, and special characters
    cleaned = re.sub(r'\(.*?\)|[A-Z]{3}\d{3}|[^a-zA-Z\s]', '', course_name)

    # Tokenize and filter
    words = [ word.upper() for word in cleaned.split()
              if word.lower() not in {'and','of'} and len(word) > 0
            ]

    # Create abbreviation from first letters
    abbreviation = ''.join([word[0] for word in words if not word.isnumeric()])
    return abbreviation if len(abbreviation) >= 2 else None

def process_session_numbers(course, entity_data):
    sessions = set()
    for session in course.get("session_plan", []):
        session_num = str(session.get("Session", "")).strip()
        if session_num:
            sessions.add(session_num)
    entity_data['SessionNumber']['values'].update(sessions)

def process_course(course, entity_data, all_assessments, all_instructors, course_properties):
    metadata = course.get("course_metadata", {})
    program = course.get("program", "").strip()
    term = course.get("term", "").strip()
    course_name = metadata.get("Course Code and Course Title", "Unknown Course").strip()

    # Generate abbreviation
    abbr = generate_abbreviation(course_name)
    synonyms = [course_name]
    if abbr:
        synonyms.extend([abbr, abbr[:2]])

    # Core entities
    entity_data['Program']['values'].add(program)
    entity_data['Term']['values'].add(term)
    entity_data['CourseInfo']['values'].add(course_name)
    entity_data['CourseInfo']['synonyms'][course_name].extend(synonyms)

    # Program-course-term relationship
    entity_data['CourseProgram']['values'].add(f"{course_name}::{program}")
    entity_data['ProgramTerm']['values'].add(f"{program}::{term}")

    # Process session numbers
    process_session_numbers(course, entity_data)

    # Course properties
    if metadata.get("Course Credit"):
        course_properties.add("credits")
    if metadata.get("Pre-requisites (if any)"):
        course_properties.add("prerequisites")
    if metadata.get("Session Duration"):
        course_properties.add("session duration")

    # Assessment relationships
    for assessment in course.get("assessment", []):
        tool = assessment.get("Assessment Tool", "").strip()
        if tool:
            all_assessments.add(tool)
            entity_data['CourseAssessment']['values'].add(f"{course_name}::{tool}")

    # Instructor relationships
    instructor = course.get("instructor_details", {}).get("Instructor(s)", "").strip()
    if instructor:
        all_instructors.add(instructor)
        entity_data['CourseInstructor']['values'].add(f"{course_name}::{instructor}")
        entity_data['InstructorProgram']['values'].add(f"{instructor}::{program}")

def create_entities(entities_dir, entity_data, all_assessments, all_instructors, course_properties):
    # Core Entities
    entities = {
        'CourseInfo': {
            'entries': [{"value": name,
                         "synonyms": list(set([name] + entity_data['CourseInfo']['synonyms'].get(name, [])))
                         } for name in entity_data['CourseInfo']['values']],
            'config': {"isEnum": False, "automatedExpansion": True}
        },
        'SessionNumber': {
            'entries': [{"value": sn, "synonyms": []} for sn in sorted(entity_data['SessionNumber']['values'],
                                                                       key=lambda x: int(re.sub(r'\D', '', x)) if re.search(r'\d', x) else 0)],
            'config': {"isEnum": True}
        },
        'Program': {
            'entries': [{"value": p, "synonyms": []} for p in entity_data['Program']['values']],
            'config': {"isEnum": True}
        },
        'Term': {
            'entries': [{"value": t, "synonyms": []} for t in entity_data['Term']['values']],
            'config': {"isEnum": True}
        },
        'AssessmentTool': {
            'entries': [{"value": a, "synonyms": []} for a in all_assessments],
            'config': {"isEnum": True}
        },
        'Instructor': {
            'entries': [{"value": i, "synonyms": []} for i in all_instructors],
            'config': {"isEnum": True}
        },
        'CourseProperty': {
            'entries': [{"value": p, "synonyms": []} for p in course_properties],
            'config': {"isEnum": True}
        },
        # Relationship-based Entities
        'CourseProgram': {
            'entries': [{"value": cp, "synonyms": []} for cp in entity_data['CourseProgram']['values']],
            'config': {"isEnum": True}
        },
        'ProgramTerm': {
            'entries': [{"value": pt, "synonyms": []} for pt in entity_data['ProgramTerm']['values']],
            'config': {"isEnum": True}
        }
    }

    for entity_name, data in entities.items():
        save_entity(entities_dir, entity_name, data['config'], data['entries'])

def save_entity(entities_dir, name, config, entries):
    entity_path = os.path.join(entities_dir, f"{name}.json")
    entries_path = os.path.join(entities_dir, f"{name}_entries_en.json")

    with open(entity_path, "w") as f:
        json.dump({
            "id": str(uuid.uuid4()),
            "name": name,
            **config
        }, f, indent=2)

    with open(entries_path, "w") as f:
        json.dump(entries, f, indent=2)

def create_intents(intents_dir):
    # Ontology-aware intents for all 30 queries
    intents = [
    #Course Metadata
        {
            "name": "GetCourseCredits",
            "templates": [
                ["How many credits is ", "@CourseInfo", " worth?"],
                ["What is the credit value of ", "@CourseInfo", "?"],
                ["Credits for ", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetCourseType",
            "templates": [
                ["Is", "@CourseInfo", "a core or elective course?"],
                ["Course type of", "@CourseInfo"],
                ["Is", "@CourseInfo", "mandatory?"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetPrerequisites",
            "templates": [
                ["What are the prerequisites for", "@CourseInfo", "?"],
                ["Do I need any prior knowledge for", "@CourseInfo", "?"],
                ["Requirements to take", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetSessionDuration",
            "templates": [
                ["How long is each session for", "@CourseInfo", "?"],
                ["Duration of sessions in", "@CourseInfo"],
                ["Session length for", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetTotalSessions",
            "templates": [
                ["How many sessions does", "@CourseInfo", "have?"],
                ["Total sessions in", "@CourseInfo", "course"],
                ["Number of classes for", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },

    #Basic Info
        {
            "name": "GetCourseOverview",
            "templates": [
                ["Overview of", "@CourseInfo"],
                ["What is", "@CourseInfo", "about?"],
                ["Introduction to", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetLearningOutcomes",
            "templates": [
                ["What are the learning outcomes for", "@CourseInfo", "?"],
                ["What will I learn in", "@CourseInfo", "?"],
                ["Learning objectives of", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetPedagogy",
            "templates": [
                ["What teaching methods are used in", "@CourseInfo", "?"],
                ["Pedagogy of", "@CourseInfo"],
                ["How is", "@CourseInfo", "taught?"]
            ],
            "params": ["CourseInfo"]
        },

    #Instructor Info
        {
            "name": "GetInstructorForCourse",
            "templates": [
                ["Who teaches ", "@CourseInfo", "?"],
                ["Instructor for ", "@CourseInfo"],
                ["Who is the professor for ", "@CourseInfo", "?"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetInstructorContact",
            "templates": [
                ["Contact details of", "@CourseInfo"],
                ["What are the contact details for the instructor of", "@CourseInfo", "?"],
                ["How can I reach the instructor for", "@CourseInfo", "?"],
                ["I need the contact information for the instructor of", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetContactInstructor",
            "templates": [
                ["Contact details of", "@Instructor"],
                ["How to contact", "@Instructor", "?"],
                ["@Instructor", "email or phone number"],
                ["Office of", "@Instructor"],
                ["Consultation hours of", "@Instructor"],
                ["What is the contact info for", "@Instructor", "?"],
                ["Give me the contact information for", "@Instructor"],
                ["How can I reach", "@Instructor", "for consultation?"]
            ],
            "params": ["Instructor"]
        },
        {
            "name": "GetCoursesByInstructor",
            "templates": [
                ["What courses does", "@Instructor", "teach?"],
                ["Classes taught by", "@Instructor"],
                ["Subjects handled by", "@Instructor"]
            ],
            "params": ["Instructor"]
        },

    #Sessions and Topics
        {
            "name": "GetCourseTopics",
            "templates": [
                ["Topics covered in", "@CourseInfo"],
                ["What topics are taught in", "@CourseInfo", "?"],
                ["Curriculum of", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetSessionInfo",
            "templates": [
                ["Session", "@SessionNumber", "of", "@CourseInfo"],
                ["What's covered in", "@CourseInfo", "session", "@SessionNumber", "?"],
                ["@CourseInfo", "class", "@SessionNumber", "details"],
                ["Provide details for session", "@SessionNumber", "of", "@CourseInfo"],
                ["Give me info on session", "@SessionNumber", "in", "@CourseInfo"],
                ["What's happening in session", "@SessionNumber", "of", "@CourseInfo", "?"],
                ["Session", "@SessionNumber", "details for", "@CourseInfo", "please"],
                ["Outline session", "@SessionNumber", "of", "@CourseInfo"]
            ],
            "params": ["CourseInfo", "SessionNumber"]
        },
        {
            "name": "GetFullSessionPlan",
            "templates": [
                ["Full session plan for", "@CourseInfo"],
                ["Detailed schedule of", "@CourseInfo"],
                ["Complete session outline for", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },

    #Assessments
        {
            "name": "GetAssessmentTools",
            "templates": [
                ["Assessment tools for", "@CourseInfo"],
                ["What assessments are in", "@CourseInfo", "?"],
                ["Grading structure of", "@CourseInfo"],
                ["List the assessment methods for", "@CourseInfo"],
                ["What evaluation tools does", "@CourseInfo", "use?"],
                ["How is", "@CourseInfo", "graded?"],
                ["Show the assessment components of", "@CourseInfo"],
                ["Which assessment techniques are applied in", "@CourseInfo", "?"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetAssessmentPercentage",
            "templates": [
                ["Percentage of", "@AssessmentTool", "in", "@CourseInfo"],
                ["Weight of", "@AssessmentTool", "in", "@CourseInfo"],
                ["How much does", "@AssessmentTool", "contribute to", "@CourseInfo"],
                ["What is the weightage of", "@AssessmentTool", "in", "@CourseInfo", "?"],
                ["How significant is", "@AssessmentTool", "in", "@CourseInfo", "grading?"],
                ["Indicate the percentage contribution of", "@AssessmentTool", "for", "@CourseInfo"],
                ["Calculate the percentage for", "@AssessmentTool", "in", "@CourseInfo"],
                ["Determine the weight of", "@AssessmentTool", "in", "@CourseInfo"]
            ],
            "params": ["CourseInfo", "AssessmentTool"]
        },
        {
            "name": "GetAssessmentDetails",
            "templates": [
                ["Detailed assessment breakdown for", "@CourseInfo"],
                ["Assessment structure of", "@CourseInfo"],
                ["Grading components in", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetHighestAssessmentTool",
            "templates": [
                ["Which assessment has the highest weight in", "@CourseInfo", "?"],
                ["Top weighted tool in", "@CourseInfo"],
                ["Most important assessment in", "@CourseInfo"],
                ["Identify the top weighted assessment in", "@CourseInfo"],
                ["Find out which assessment is most significant in", "@CourseInfo"],
                ["Show the assessment with the highest score in", "@CourseInfo"],
                ["Reveal the assessment tool with maximum weight in", "@CourseInfo"],
                ["Which evaluation has the greatest impact in", "@CourseInfo", "?"]
            ],
            "params": ["CourseInfo"]
        },

    #Additional Course Metadata queries
        {
            "name": "GetYearBatch",
            "templates": [
                ["Which batch is", "@CourseInfo", "offered to?"],
                ["Year eligibility for", "@CourseInfo"],
                ["When is", "@CourseInfo", "taught?"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetSections",
            "templates": [
                ["Sections available for", "@CourseInfo"],
                ["Which sections offer", "@CourseInfo", "?"],
                ["Groups for", "@CourseInfo"]
            ],
            "params": ["CourseInfo"]
        },

    #Program/Term related queries
        {
            "name": "GetProgramForCourse",
            "templates": [
                ["Which program offers", "@CourseInfo", "?"],
                ["In which program is", "@CourseInfo", "taught?"],
                ["@CourseInfo", "program information"],
                ["Which program includes", "@CourseInfo", "?"],
                ["Find the program where", "@CourseInfo", "is available"],
                ["Tell me the program for", "@CourseInfo"],
                ["What is the program that hosts", "@CourseInfo", "?"],
                ["In which academic program is", "@CourseInfo", "offered?"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetTermForCourseProgram",
            "templates": [
                ["Which term is @CourseInfo conducted for @Program?"],
                ["For @Program, which term includes @CourseInfo?"],
                ["Term details for @CourseInfo in @Program"],
                ["In which term is @CourseInfo taught under @Program?"],
                ["Find the term for @CourseInfo in @Program program"],
                ["What term features", "@CourseInfo", "in", "@Program", "?"],
                ["Identify the term in which", "@CourseInfo", "is offered for", "@Program"],
                ["Please specify the term for", "@CourseInfo", "in", "@Program", "program"]
            ],
            "params": ["CourseInfo", "Program"]
        },
        {
            "name": "GetCoursesInProgramTerm",
            "templates": [
                ["Courses in", "@Program", "program"],
                ["What courses are offered in", "@Program", "during", "@Term", "?"],
                ["List", "@Program", "courses"],
                ["List all courses under", "@Program", "in", "@Term"],
                ["What are the courses for", "@Program", "?"],
                ["Show me the courses for", "@Program", "program in", "@Term"],
                ["Which classes are part of", "@Program", "during", "@Term", "?"],
                ["What are the available courses in", "@Program", "for", "@Term", "?"]
            ],
            "params": ["Program", "Term"]
        },
        {
            "name": "GetInstructorsInProgramTerm",
            "templates": [
                ["Instructors teaching in", "@Program", "during", "@Term"],
                ["Faculty for", "@Program", "term", "@Term"],
                ["Who teaches in", "@Program", "in", "@Term", "?"]
            ],
            "params": ["Program", "Term"]
        },

    #Instructor additional queries
        {
            "name": "GetInstructorOffice",
            "templates": [
                ["Office location of the instructor for", "@CourseInfo"],
                ["Where is the office of the professor for", "@CourseInfo", "?"],
                ["Office address for", "@CourseInfo", "instructor"]
            ],
            "params": ["CourseInfo"]
        },
        {
            "name": "GetCoursesByInstructorInProgram",
            "templates": [
                ["Courses taught by", "@Instructor"],
                ["@Instructor", "teaching schedule"],
                ["Which courses does", "@Instructor", "teach in", "@Program", "?"],
                ["Which courses does", "@Instructor", "handle?"],
                ["Tell me the courses of", "@Instructor", "in", "@Program"],
                ["List all classes by", "@Instructor", "in", "@Program"],
                ["What subjects does", "@Instructor", "teach in", "@Program", "?"],
                ["Give me the teaching schedule of", "@Instructor", "for", "@Program"]
            ],
            "params": ["Instructor", "Program"]
        },
        {
            "name": "GetConsultationContact",
            "templates": [
                ["Consultation hours for the instructor of", "@CourseInfo"],
                ["When can I meet the professor for", "@CourseInfo", "?"],
                ["Office hours for", "@CourseInfo", "teacher"]
            ],
            "params": ["CourseInfo"]
        },
    ]

    for intent in intents:
        create_intent(
            intents_dir=intents_dir,
            name=intent["name"],
            templates=intent["templates"],
            params=intent["params"]
        )

def create_intent(intents_dir, name, templates, params):
    intent_json = {
        "id": str(uuid.uuid4()),
        "name": name,
        "auto": True,
        "contexts": [],
        "responses": [{
            "resetContexts": False,
            "action": "",
            "affectedContexts": [],
            "parameters": [{
                "name": "courseName" if p == "CourseInfo"
                  else "sessionNumber" if p == "SessionNumber"
                  else "assessmentTool" if p == "AssessmentTool"
                  else "instructorName" if p == "Instructor"
                  else "courseProperty" if p == "CourseProperty"
                  else p.lower(),
                "displayName": p,
                "value": f"${p}",
                "meta":f"@{p}",
                "entityTypeDisplayName": f"{p}",
                "isList": False,
                "prompts": [],
                "mandatory": True if p == "CourseInfo" else False
            } for p in params],
            "messages": [{
                "type": 0,
                "lang": "en",
                "speech": ["Fetching data from university knowledge base..."]
            }]
        }],
        "webhookUsed": True,
        "webhookForSlotFilling": False,
        "fallbackIntent": False,
        "events": [],
        "conditionalResponses": []
    }

    usersays = []
    for template in templates:
        data = []
        for item in template:
            if item.startswith("@"):
                entity = item[1:]
                data.append({
                    "text": f"{{{entity}}}",
                    "alias": "courseName" if entity == "CourseInfo"
                  else "sessionNumber" if entity == "SessionNumber"
                  else "assessmentTool" if entity == "AssessmentTool"
                  else "instructorName" if entity == "Instructor"
                  else "courseProperty" if entity == "CourseProperty"
                  else entity.lower(),
                    "meta": f"@{entity}",
                    "userDefined": True
                    # "text": f"{{{entity}}}",
                    # "alias": entity.lower(),
                    # "meta": f"@{entity}",
                    # "userDefined": True
                })
            else:
                data.append({"text": item, "userDefined": False})

        usersays.append({
            "id": str(uuid.uuid4()),
            "data": data,
            "isTemplate": False,
            "count": 0,
            "updated": int(datetime.now().timestamp())
        })

    intent_path = os.path.join(intents_dir, f"{name}.json")
    usersays_path = os.path.join(intents_dir, f"{name}_usersays_en.json")

    with open(intent_path, "w") as f:
        json.dump(intent_json, f, indent=2)

    with open(usersays_path, "w") as f:
        json.dump(usersays, f, indent=2)

def create_metadata(temp_dir):
    # agent.json
    with open(os.path.join(temp_dir, "agent.json"), "w") as f:
        json.dump({
            "description": "ChatBot for course related information",
            "language": "en",
            "webhook": {
                "url": "https://39a1-112-196-38-60.ngrok-free.app/webhook",
                "available": True,
                "useForDomains": True
            },
            "defaultTimezone": "Asia/Colombo",
            "mlMinConfidence": 0.3,
            "supportedLanguages": ["en"],
            "onePlatformApiVersion": "v2",
        }, f, indent=2)

    # package.json
    with open(os.path.join(temp_dir, "package.json"), "w") as f:
        json.dump({"version": "2.1"}, f, indent=2)

if __name__ == "__main__":
    input_folder = "/content/drive/MyDrive/Outputs"
    output_folder = "/content/drive/MyDrive/Intents"
    generate_dialogflow_agent(input_folder, output_folder)

Agent package created: /content/drive/MyDrive/Intents/CourseSensei(v2.1)_$intents.zip
